In [110]:
percent_of_test_data = 30
main_url = 'https://www.otomoto.pl/osobowe/volkswagen/passat/b6-2005-2010/?search%5Bfilter_enum_fuel_type%5D%5B0%5D=diesel&search%5Bfilter_enum_damaged%5D=0&search%5Bcountry%5D='

In [69]:
import pandas as pd
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from bs4 import BeautifulSoup

In [70]:
#with urllib.request.urlopen(url) as response:
html = urllib.request.urlopen(main_url).read()
soup = BeautifulSoup(html,"lxml")

pages = int(soup.find_all('span', attrs={'class': 'page'})[-2].text)
print('Found pages: ',pages)

Found pages:  50


In [97]:
Xy = np.empty((0, 4))

for page in range(1,10): #pages+1):
    url = main_url+'&page='+str(page)    
    html = html+urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html,"lxml")
    
    articles = soup.find_all('article')
    print('Page: '+str(page)+' | Offers: '+str(len(articles)))

    for article in articles:        
        #year
        year_tag = article.find("li", attrs={'data-code': 'year'})
        if (year_tag == None):
            continue
        year = int(year_tag.text)        
        #mileage
        mileage_tag = article.find("li", attrs={'data-code': 'mileage'})
        if (mileage_tag == None):
            continue
        mileage = int(mileage_tag.span.text.replace(' ', '')[:-2])
        # capacity
        capacity_tag = article.find("li", attrs={'data-code': 'engine_capacity'})
        if (capacity_tag == None):
            continue
        capacity = int(capacity_tag.span.text.replace(' ', '')[:-3])
        if capacity<1896: capacity = 1598
        elif capacity >= 1896 and capacity < 1968: capacity =  1896
        else: capacity = 1968             
        # price
        price_tag = article.find("span", attrs={'class': 'offer-price__number'})
        if (price_tag == None):
            continue
        price = int(price_tag.contents[0].replace(' ', ''))
        Xy = np.append(Xy, [[year, mileage, capacity, price]], axis=0)

Page: 1 | Offers: 32


Page: 2 | Offers: 32


Page: 3 | Offers: 32


Page: 4 | Offers: 32


Page: 5 | Offers: 32


...

Page: 35 | Offers: 32


Page: 36 | Offers: 32


Page: 37 | Offers: 32


Page: 38 | Offers: 32


Page: 39 | Offers: 32


In [98]:
n = len(Xy)
print('\nTotal samples: ',n)

#print(Xv[-10:])
np.random.shuffle(Xy)

n_train = round((100 - percent_of_test_data) / 100 * n)
n_test = n - n_train

[Xy_train, Xy_test, _] = np.split(Xy, [n_train, n_train + n_test])

print('Training samples: %d' % len(Xy_train))
print('Test samples: %d' % len(Xy_test))

X_train = Xy_train[:,[0,1,2]]
y_train = Xy_train[:,[3]]

X_test = Xy_test[:,[0,1,2]]
y_test = Xy_test[:,[3]]

#print(X.shape)
#print(y.shape)


Total samples:  1170
Training samples: 819
Test samples: 351


In [108]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
print('Coefficients: \n', regr.coef_)
regr.intercept_

Coefficients: 
 [[  2.22518100e+03  -2.69006247e-02   6.13379556e-01]]


array([-4439030.37977308])

In [100]:
y_pred = regr.predict(X_test)

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 5821733.32
Variance score: 0.70


In [106]:
price_pred = regr.predict([[2010,180000, 1968]])
print(str(round(price_pred[0][0],2))+' PLN')

29948.45 PLN
